In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')
import geemap

masalia_boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/jharkhand/dumka/masalia/lulc_v4_dumka_masalia_boundaries")
masalia_roi = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/jharkhand/dumka/masalia/filtered_mws_dumka_masalia_uid")

import re
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').select('predicted_label')
PARENT = 'projects/raman-461708/assets'

# ---- List all image assets once (fast follow-up lookups) ----
def _basename(asset_name: str) -> str:
    return asset_name.split('/')[-1]

all_assets = ee.data.listAssets({'parent': PARENT}).get('assets', [])
image_assets = [a for a in all_assets if a.get('type','').lower() == 'image']
basenames = {_basename(a['name']) for a in image_assets}

def asset_exists_base(base: str) -> bool:
    return base in basenames

def fullpath(base: str) -> str:
    return f'{PARENT}/{base}'

def list_part_bases_for_aez(aez_no: int):
    """Return sorted basenames for all parts like AEZ_{n}_v4_local_partK (K numeric)."""
    rx = re.compile(rf'^AEZ_{aez_no}_v4_local_part(\d+)$')
    parts = []
    for b in basenames:
        m = rx.match(b)
        if m:
            parts.append((int(m.group(1)), b))
    # Sort by numeric suffix to keep deterministic order (not required for mosaic, but nice to have).
    parts.sort(key=lambda x: x[0])
    return [b for _, b in parts]

def choose_assets_for_aez(aez_no: int):
    """
    Preference:
      1) AEZ_{n}_v4
      2) AEZ_{n}_v4_local_part*
      3) AEZ_{n}_v4_local
    Returns: list of full asset IDs (possibly multiple if parts), plus a metadata dict.
    """
    base_v4 = f'AEZ_{aez_no}_v4'
    base_local = f'AEZ_{aez_no}_v4_local'
    part_bases = list_part_bases_for_aez(aez_no)

    if asset_exists_base(base_v4):
        return [fullpath(base_v4)], {'source':'v4','parts':[]}

    if part_bases:
        return [fullpath(b) for b in part_bases], {'source':'v4_local_parts','parts':part_bases}

    if asset_exists_base(base_local):
        return [fullpath(base_local)], {'source':'v4_local','parts':[]}

    return [], {'source':'missing','parts':[]}

# ---- Collect images AEZ 1..19 ----
images = []
report = {}   # aez_no -> {'source':..., 'parts':[...] }
missing = []

for aez_no in range(1, 20):
    asset_ids, meta = choose_assets_for_aez(aez_no)
    report[aez_no] = meta
    if asset_ids:
        # If this AEZ has multiple parts, add them all; mosaic will merge them.
        for aid in asset_ids:
            images.append(ee.Image(aid))
    else:
        missing.append(aez_no)

if not images:
    raise RuntimeError("No AEZ images found across v4 / v4_local_parts / v4_local")

# ---- Final mosaic across all gathered images ----
# Order: AEZ 1..19; within an AEZ, parts in ascending numeric order.
# ImageCollection.mosaic() draws later images on top of earlier ones.
mosaic_img = ee.ImageCollection(images).mosaic()
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}
aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

m.addLayer(mosaic_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(aez, {}, "AEZ boundaries")
m.addLayer(masalia_roi, {}, "Masalia ROI")
m.addLayer(masalia_boundaries, {}, "Masalia Boundaries")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [2]:
from pathlib import PurePosixPath

ROOT = 'projects/ee-corestackdev/assets/apps/mws'  # mws root

def list_children(parent):
    """Generator over immediate children of an EE asset folder, with paging."""
    page_token = None
    while True:
        resp = ee.data.listAssets({'parent': parent, 'pageSize': 1000, 'pageToken': page_token})
        for a in resp.get('assets', []):
            yield a
        page_token = resp.get('nextPageToken')
        if not page_token:
            break

def walk_folders(root):
    """Depth-first traversal over folders under root, yielding every asset."""
    stack = [root]
    while stack:
        parent = stack.pop()
        for a in list_children(parent):
            typ = a.get('type', '')
            name = a.get('name')  # full path like 'projects/.../assets/apps/mws/...'
            if typ.upper() == 'FOLDER':
                stack.append(name)
            else:
                yield a

def parse_mws_path(full_name):
    """
    Parse paths like:
    projects/ee-corestackdev/assets/apps/mws/<state>/<district>/<block>/filtered_mws_<state>_<block>_uid
    Return (state, district, block, basename) or None if it doesn't fit.
    """
    # Normalize to the part after /assets/
    try:
        # Split at '/assets/' to get the logical asset path
        logical = full_name.split('/assets/', 1)[1]
    except IndexError:
        return None
    parts = PurePosixPath(logical).parts  # ('apps','mws','<state>','<district>','<block>','filtered_mws_...')

    if len(parts) < 6 or parts[0] != 'apps' or parts[1] != 'mws':
        return None

    state, district, block = parts[2], parts[3], parts[4]
    basename = parts[-1]

    # Must start with filtered_mws_ and end with _uid and match state/block in name
    if not (basename.startswith('filtered_mws_') and basename.endswith('_uid')):
        return None

    # Expect exactly: filtered_mws_<state>_<block>_uid
    expected = f'filtered_mws_{district}_{block}_uid'
    if basename != expected:
        return None

    return state, district, block, basename

def find_filtered_mws_assets(root=ROOT):
    matches = []
    for asset in walk_folders(root):
        name = asset.get('name')  # canonical path
        print(name)
        parsed = parse_mws_path(name)
        if parsed:
            state, district, block, basename = parsed
            matches.append({
                'full_path': name,
                'state': state,
                'district': district,
                'block': block,
                'asset_type': asset.get('type'),
            })
    return matches

results = find_filtered_mws_assets(ROOT)

projects/ee-corestackdev/assets/apps/mws/indian_state_boundaries
projects/ee-corestackdev/assets/apps/mws/west_bengal/west_bengal_lithology
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/ET_annual_south_twenty-four_parganas_kakdwip
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/ET_fortnight_south_twenty-four_parganas_kakdwip
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/ET_fortnight_south_twenty-four_parganas_kakdwip_2017_2022
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/ET_fortnight_south_twenty-four_parganas_kakdwip_2022_2024
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/Prec_annual_south_twenty-four_parganas_kakdwip
projects/ee-corestackdev/assets/apps/mws/west_bengal/south_twenty-four_parganas/kakdwip/Prec_fortnight_south_twenty-four_parganas_kakdwip
projects/ee-corestackdev/a

In [3]:
results_pats = [i["full_path"] for i in results]

In [8]:
merged_fc = []
# Start with an empty placeholder
merged_fc = ee.FeatureCollection([])

for i, path in enumerate(results_pats, 1):
    fc = ee.FeatureCollection(path)

    # Optionally tag each feature with its source path
    fc = fc.map(lambda f: f.set('source_path', path))

    # Merge with the growing collection
    merged_fc = merged_fc.merge(fc)

In [9]:
m.addLayer(merged_fc, {}, "All MWS filtered UIDs")
m 

Map(bottom=919586.0, center=[21.70017946130734, 85.37243843078615], controls=(WidgetControl(options=['position…

In [ ]:
source_path: projects/ee-corestackdev/assets/apps/mws/karnataka/chik_ballapur/chintamani/filtered_mws_chik_ballapur_chintamani_uid


In [4]:
# Threshold rule: use > 0.5 for "more than 50%". Change to gte if you want ">= 50%".
USE_STRICT_GT = True  # True => > 0.5 ; False => >= 0.5

# Class groups (by your schema)
FARM_CLASSES = [8, 9, 10, 11]
SCRUB_CLASSES = [12, 7]
FOREST_CLASS  = [6]
PLANTATION    = [13]

# ---- INIT ----
ee.Initialize()

lulc = mosaic_img.select("predicted_label").clip(masalia_roi).rename('lulc').toInt()
bounds = masalia_boundaries.filter(ee.Filter.neq("class", "plantation"))

# Ensure we only count pixels where LULC exists (mask area by LULC mask)
area_img = ee.Image.pixelArea().updateMask(lulc.mask()).rename('px_area')



# Reduce per polygon: sum(pixel area) grouped by LULC class
grouped = area_img.addBands(lulc).reduceRegions(
    collection=bounds,
    reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),  # band 1 = 'lulc' (area is band 0)
    scale=10
)

def list_to_area_dict(group_list):
    group_list = ee.List(group_list)
    empty = ee.Dictionary({})
    def _acc(el, acc):
        el = ee.Dictionary(el)
        cls = ee.Number(el.get('class')).format()
        sm  = ee.Number(el.get('sum'))
        return ee.Dictionary(acc).set(cls, sm)
    return ee.Dictionary(group_list.iterate(_acc, empty))

def frac_for(f_area_dict, cls_ids, poly_area):
    cls_ids = ee.List(cls_ids)
    total = ee.Number(cls_ids.iterate(
        lambda c, acc: ee.Number(acc).add(ee.Number(f_area_dict.get(ee.Number(c).format(), 0))),
        0
    ))
    return ee.Algorithms.If(ee.Number(poly_area).gt(0), ee.Number(total).divide(poly_area), 0)

def classify_feature(f):
    f = ee.Feature(f)
    poly_area = f.geometry().area(maxError=1)

    # SAFE way to get the reducer output list `groups`
    raw_groups = f.get('groups')  # may be null
    groups = ee.List(ee.Algorithms.If(
        ee.Algorithms.IsEqual(raw_groups, None),
        ee.List([]),
        raw_groups
    ))

    area_dict = list_to_area_dict(groups)

    farm_frac   = ee.Number(frac_for(area_dict, [8,9,10,11], poly_area))
    scrubs_frac = ee.Number(frac_for(area_dict, [12,7],      poly_area))
    forest_frac = ee.Number(frac_for(area_dict, [6],         poly_area))
    plant_frac  = ee.Number(frac_for(area_dict, [13],        poly_area))

    thr = 0.5  # change to >= by swapping gt->gte
    label = ee.Algorithms.If(farm_frac.gt(thr),   'farm',
             ee.Algorithms.If(scrubs_frac.gt(thr),'scrubs',
             ee.Algorithms.If(forest_frac.gt(thr),'forest',
             ee.Algorithms.If(plant_frac.gt(thr), 'plantation', 'mixed'))))

    return f.set({
        'farm_frac': farm_frac,
        'scrubs_frac': scrubs_frac,
        'forest_frac': forest_frac,
        'plantation_frac': plant_frac,
        'label': label
    })


classified = grouped.map(classify_feature)

In [5]:
classified.limit(1).getInfo()

KeyboardInterrupt: 

In [ ]:
parts = lulc.reduceRegions(
    collection=bounds,
    reducer=ee.Reducer.frequencyHistogram(),  # returns a {class: count} dict
    scale=10
)

In [ ]:
parts.limit(10).getInfo()

In [3]:
FARM_CLASSES = [8, 9, 10, 11]
SCRUB_CLASSES = [12, 7]
FOREST_CLASS  = [6]
PLANTATION    = [13]


lulc = mosaic_img.select("predicted_label").clip(masalia_roi).rename('lulc').toInt()
bounds = masalia_boundaries.filter(ee.Filter.neq("class", "plantation"))

def get_distribution(feature):
    feature = ee.Feature(feature)
    hist = lulc.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=feature.geometry(),
        scale=10)
    
    # 2) Pull {class_str: count} from the 'lulc' band
    counts = ee.Dictionary(hist.get('lulc', ee.Dictionary({})))

    # 3) Total pixels inside this boundary
    total = ee.Number(
        ee.List(counts.values()).iterate(
            lambda v, acc: ee.Number(acc).add(ee.Number(v)),
            0
        )
    )

    # 4) Sum helper for a list of numeric class IDs (convert each to string key)
    def sum_classes(class_list):
        class_list = ee.List(class_list)
        def _acc(c, acc):
            key = ee.Number(c).format()                  # convert 8 -> "8"
            val = ee.Number(counts.get(key, 0))          # counts have string keys
            return ee.Number(acc).add(val)
        return ee.Number(class_list.iterate(_acc, 0))

    # 5) Compute farm % = classes 8,9,10,11 / total
    farm_count = sum_classes(FARM_CLASSES)
    plantation_count = sum_classes(PLANTATION)
    
    farm_pct = ee.Algorithms.If(ee.Number(total).gt(0), ee.Number(farm_count).divide(total).multiply(100), 0)
    plantation_pct = ee.Algorithms.If(ee.Number(total).gt(0), ee.Number(plantation_count).divide(total).multiply(100), 0)
    
    return feature.set('plantation_pct', farm_pct)


boundaries = bounds.map(get_distribution)

In [23]:
boundaries.filter(ee.Filter.gt('plantation_pct', 50)).limit(10)

KeyboardInterrupt: 

In [4]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')
import geemap

masalia_boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/jharkhand/dumka/masalia/lulc_v4_dumka_masalia_boundaries")
masalia_roi = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/jharkhand/dumka/masalia/filtered_mws_dumka_masalia_uid")

import re
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').select('predicted_label')
PARENT = 'projects/raman-461708/assets'

# ---- List all image assets once (fast follow-up lookups) ----
def _basename(asset_name: str) -> str:
    return asset_name.split('/')[-1]

all_assets = ee.data.listAssets({'parent': PARENT}).get('assets', [])
image_assets = [a for a in all_assets if a.get('type','').lower() == 'image']
basenames = {_basename(a['name']) for a in image_assets}

def asset_exists_base(base: str) -> bool:
    return base in basenames

def fullpath(base: str) -> str:
    return f'{PARENT}/{base}'

def list_part_bases_for_aez(aez_no: int):
    """Return sorted basenames for all parts like AEZ_{n}_v4_local_partK (K numeric)."""
    rx = re.compile(rf'^AEZ_{aez_no}_v4_local_part(\d+)$')
    parts = []
    for b in basenames:
        m = rx.match(b)
        if m:
            parts.append((int(m.group(1)), b))
    # Sort by numeric suffix to keep deterministic order (not required for mosaic, but nice to have).
    parts.sort(key=lambda x: x[0])
    return [b for _, b in parts]

def choose_assets_for_aez(aez_no: int):
    """
    Preference:
      1) AEZ_{n}_v4
      2) AEZ_{n}_v4_local_part*
      3) AEZ_{n}_v4_local
    Returns: list of full asset IDs (possibly multiple if parts), plus a metadata dict.
    """
    base_v4 = f'AEZ_{aez_no}_v4'
    base_local = f'AEZ_{aez_no}_v4_local'
    part_bases = list_part_bases_for_aez(aez_no)

    if asset_exists_base(base_v4):
        return [fullpath(base_v4)], {'source':'v4','parts':[]}

    if part_bases:
        return [fullpath(b) for b in part_bases], {'source':'v4_local_parts','parts':part_bases}

    if asset_exists_base(base_local):
        return [fullpath(base_local)], {'source':'v4_local','parts':[]}

    return [], {'source':'missing','parts':[]}

# ---- Collect images AEZ 1..19 ----
images = []
report = {}   # aez_no -> {'source':..., 'parts':[...] }
missing = []

for aez_no in range(1, 20):
    asset_ids, meta = choose_assets_for_aez(aez_no)
    report[aez_no] = meta
    if asset_ids:
        # If this AEZ has multiple parts, add them all; mosaic will merge them.
        for aid in asset_ids:
            images.append(ee.Image(aid))
    else:
        missing.append(aez_no)

if not images:
    raise RuntimeError("No AEZ images found across v4 / v4_local_parts / v4_local")

# ---- Final mosaic across all gathered images ----
# Order: AEZ 1..19; within an AEZ, parts in ascending numeric order.
# ImageCollection.mosaic() draws later images on top of earlier ones.
mosaic_img = ee.ImageCollection(images).mosaic()
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}
aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

m.addLayer(mosaic_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(aez, {}, "AEZ boundaries")
m.addLayer(masalia_roi, {}, "Masalia ROI")
m.addLayer(boundaries.filter(ee.Filter.gt('farm_pct', 50)), {}, "Masalia Boundaries")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [4]:
result = boundaries.filter(ee.Filter.gt('farm_pct', 50))
result = boundaries.filter(ee.Filter.gt('plantation_pct', 50))

In [5]:
# Choose the properties you want in the export
EXPORT_PROPS = [
    'class',
    'plantation_pct',
    # add any original boundary attributes you want to keep, e.g.:
    # 'id', 'name', 'village', ...
]

# Build a slimmed FeatureCollection with only the chosen properties + geometry
def _to_slim(f):
    f = ee.Feature(f)
    props = f.toDictionary(EXPORT_PROPS)  # only the ones we want
    return ee.Feature(f.geometry(), props)

slim = result.map(_to_slim)

In [26]:
# Example asset path (new-style): projects/<your-project>/assets/<folder>/<name>
asset_id = 'projects/raman-461708/assets/masalia_plantation_boundaries'

task_asset = ee.batch.Export.table.toAsset(
    collection=slim,
    description='masalia_boundaries_plantation_gee_asset',
    assetId=asset_id
)
task_asset.start()

In [6]:
task_geo = ee.batch.Export.table.toDrive(
    collection=slim,  # use simplified one if you made it
    description='masalia_farm_boundaries',
    folder='gee_exports',                 # change or set to None for root
    fileNamePrefix='masalia_farm_boundaries',
    fileFormat='GeoJSON')
task_geo.start()